<a href="https://colab.research.google.com/github/shlok-py/DeerHack2023-BroCode/blob/classification0/dense_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"santitshakya23","key":"08d89d3ff81a346aeccf900ffeddc880"}'}

In [ ]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 70 May  5 14:25 kaggle.json


In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir -p ~/.kaggle 

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!pwd

/content


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d antoreepjana/animals-detection-images-dataset

100% 8.92G/8.92G [05:57<00:00, 30.7MB/s]
100% 8.92G/8.92G [05:57<00:00, 26.8MB/s]


In [ ]:
! unzip animals-detection-images-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/Squirrel/013240b8dc5ed267.jpg  
  inflating: train/Squirrel/01998042bc8d9fc5.jpg  
  inflating: train/Squirrel/022a6508667c26a4.jpg  
  inflating: train/Squirrel/029dfca01a8a6c27.jpg  
  inflating: train/Squirrel/02a83d8659c234be.jpg  
  inflating: train/Squirrel/02b3690baaf911b8.jpg  
  inflating: train/Squirrel/02b7b759c7f92d7c.jpg  
  inflating: train/Squirrel/02d71b61109cbe80.jpg  
  inflating: train/Squirrel/0308ef4c641df4e5.jpg  
  inflating: train/Squirrel/031dda839ff54675.jpg  
  inflating: train/Squirrel/0346ada1fdb84d6f.jpg  
  inflating: train/Squirrel/0388c753a05557b1.jpg  
  inflating: train/Squirrel/03b2c9285da59756.jpg  
  inflating: train/Squirrel/03f767d0f5a2057d.jpg  
  inflating: train/Squirrel/04044469eb7c479e.jpg  
  inflating: train/Squirrel/041e5d593d2d4b94.jpg  
  inflating: train/Squirrel/04301655bd17cd90.jpg  
  inflating: train/Squirrel/046675ea134c2256.jpg  
  inflating: train/Squirrel/046

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 10,6
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications import EfficientNetB0
from keras.applications.densenet import DenseNet121
from tensorflow.keras.optimizers import Adam
import os

In [ ]:
train_path = "/content/drive/MyDrive/Animals-data-final-classes/"
test_path = "/content/drive"

In [ ]:
for i in os.listdir(train_path):
  print(i, "--->", len(os.listdir(os.path.join(train_path, i))))

Hippopotamus ---> 77
train ---> 8
test ---> 8


In [ ]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

for i in os.listdir(train_path):
    impath = Path(os.path.join(train_path,i)).rglob("*.jpeg")
    for img_p in impath:
        try:
            img = PIL.Image.open(img_p)
            print("success")
        except PIL.UnidentifiedImageError:
            print(img_p)
        except:
            print(img_p)

In [ ]:
import os
import glob
import random
from sklearn.model_selection import train_test_split
import shutil

# Define the root directory where your folders are located
root_dir = '/content/drive/MyDrive/Animals-data-final-classes'

# Define the subdirectories (classes) in your dataset
sub_dirs = []
for i in os.listdir(root_dir):
  sub_dirs.append(i)
# /content/drive/MyDrive/Animals-data-final-classes/Cheetah
# Define the fraction of data to use for testing
train_ratio = 0.7
test_ratio = 0.3

# Loop over each subdirectory and split the files into train and test sets
for sub_dir in sub_dirs:
    # Get the list of files in the subdirectory
    files = os.listdir(os.path.join(root_dir, sub_dir))
    
    # Shuffle the files randomly
    random.shuffle(files)
    
    # Calculate the split index
    split_idx = int(train_ratio * len(files))
    
    # Split the files into train and test sets
    train_files = files[:split_idx]
    test_files = files[split_idx:]
    
    # Move the train files to a subdirectory named 'train'
    train_dir = os.path.join(root_dir,'train', sub_dir)
    os.makedirs(train_dir, exist_ok=True)
    for file in train_files:
        src_path = os.path.join(root_dir, sub_dir, file)
        dst_path = os.path.join(train_dir, file)
        shutil.move(src_path, dst_path)
    
    # Move the test files to a subdirectory named 'test'
    test_dir = os.path.join(root_dir,'test', sub_dir)
    os.makedirs(test_dir, exist_ok=True)
    for file in test_files:
        src_path = os.path.join(root_dir, sub_dir, file)
        dst_path = os.path.join(test_dir, file)
        shutil.move(src_path, dst_path)
    for i in sub_dirs:
      os.rmdir(os.path.join(root_dir,i))

OSError: ignored

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
)
train_generator = train_datagen.flow_from_directory(
    directory = "/content/drive/MyDrive/Animals-data-final-classes/train",
    target_size = (224, 224),
    batch_size = 14,
    class_mode = 'categorical'
)

Found 986 images belonging to 8 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(
    directory = "/content/drive/MyDrive/Animals-data-final-classes/test",
    target_size = (224,224),
    batch_size = 10,
    class_mode = 'categorical'
)

Found 405 images belonging to 8 classes.


In [ ]:

pretrained_model = tf.keras.applications.EfficientNetB0(include_top = False)
pretrained_model2 = DenseNet121(weights='imagenet', include_top=False, input_shape=(150,150,3))

for layer in pretrained_model2.layers:
  layer.trainable = False

inputs = layers.Input(shape = (150,150,3),name = "input_layer")
x = pretrained_model2(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(32, activation = 'relu')(x)
x = layers.Dense(8, activation = 'softmax')(x)
model = Model(inputs, x)
model.summary()

29084464/29084464 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 150, 150, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, 4, 4, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 32)                32800     
                                                                 
 dense_1 (Dense)             (None, 8)                 264       
                                                                 
Total params: 7,070,568
Trainable params: 33,064
Non-trainable

In [ ]:

lr = 0.0001
optimizer = Adam(lr=lr)
model.compile(optimizer=optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 150, 150, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, 4, 4, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 32)                32800     
                                                                 
 dense_1 (Dense)             (None, 8)                 264       
                                                                 
Total params: 7,070,568
Trainable params: 33,064
Non-trainable params: 7,037,504
______________________________________________

In [ ]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

history = model.fit(
            train_generator,
            steps_per_epoch = 50,
            epochs=3,
            validation_data=validation_generator,
            validation_steps=200,
            verbose = 1
            )

Epoch 1/3
50/50 [==============================] - ETA: 0s - loss: 1.0398 - accuracy: 0.6604

50/50 [==============================] - 373s 7s/step - loss: 1.0398 - accuracy: 0.6604 - val_loss: 0.5929 - val_accuracy: 0.8296
Epoch 2/3
50/50 [==============================] - 126s 3s/step - loss: 0.3808 - accuracy: 0.8931
Epoch 3/3
50/50 [==============================] - 121s 2s/step - loss: 0.1970 - accuracy: 0.9566


In [ ]:
model.save("model.h5")